<a href="https://colab.research.google.com/github/samiravaez/IMDB-Scraper/blob/main/imdb_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import requests
from requests import get
from bs4 import BeautifulSoup
from warnings import warn
from time import sleep
from random import randint
import numpy as np, pandas as pd
import seaborn as sns
import re
import csv

pages = np.arange(1, 100, 50)


# initialize the data structure to store the scraped content
titles = []
years = []
genres = []
runtimes = []
imdb_ratings = []
imdb_ratings_standardized = []
metascores = []
votes = []
descriptions = []
counter = 0
for page in pages:
  try:
    response = get ("https://www.imdb.com/search/title/?title_type=feature&year=2022-01-01,2022-12-31&start="
    +str(page)+"&ref_=adv_nxt")
    response.raise_for_status()
    sleep(randint(10,15))

  except requests.exceptions.RequestException as err:
    print('Request failed as:',err)

  page_imdb = BeautifulSoup(response.text,'html.parser')
  movie_containers = page_imdb.find_all('div', class_ = 'lister-item mode-advanced')

  for movieInf in movie_containers:


    #title
    if movieInf.h3.a:
      title = movieInf.h3.a.text
      titles.append(title)
    else:
      titles.append(None)

    #year released
    if movieInf.h3.find('span', class_= 'lister-item-year text-muted unbold'):
      year = movieInf.h3.find('span', class_= 'lister-item-year text-muted unbold').text
      pattern = r'^\(([^0-9]+)\)'
      match_movie_version = re.match(pattern, year)

      if match_movie_version:
        movie_version = match_movie_version.group(1)
        titles[counter] += '/'+ movie_version

      year = re.sub(pattern+'|[()]', '', year)
      if year:
       years.append(year)
      else:
       years.append(None)
    else:
      years.append(None)

    ##IMDB rating
    if movieInf.find('div', class_= 'ratings-bar').strong:
      rating = movieInf.find('div', class_= 'ratings-bar').strong.text
      imdb_ratings.append(rating)
    else:
      imdb_ratings.append(None)

    #genre
    if movieInf.p.find('span', class_ = 'genre'):
      genre = movieInf.p.find('span', class_ = 'genre').text.replace("\n", "").replace(',','|').replace(' ','') # remove the whitespace character, strip, and split to create an array of genres
      genres.append(genre)
    else:
      genres.append(None)

    #runtime
    if movieInf.p.find('span', class_ = 'runtime'):
      time = movieInf.p.find('span', class_ = 'runtime').text.replace(" min", "")
      runtimes.append(time)
    else:
      runtimes.append(time)


    #Metascore
    if movieInf.find('div', class_ = 'inline-block ratings-metascore'):
      meta_score = movieInf.find('div', class_ = 'inline-block ratings-metascore').span.text.replace(" ","")
      metascores.append(meta_score)
    else:
      metascores.append(None)

    #Number of votes
    if movieInf.find('span', attrs = {'name':'nv'})['data-value']:
      vote = movieInf.find('span', attrs = {'name':'nv'})['data-value']
      votes.append(vote)
    else:
      votes.append(None)

    # descriptions
    if movieInf.find('div', class_ = 'lister-item-content').find_all('p', class_='text-muted')[1]:
      descrip = movieInf.find('div', class_ = 'lister-item-content').find_all('p', class_='text-muted')[1].text
      descriptions.append(descrip)
    else:
      descriptions.append(None)

    counter += 1

movies2022 = pd.DataFrame({'movie': titles,
                      'year': years,
                      'rating': imdb_ratings,
                      'genre': genres,
                      'runtime': runtimes,
                      'imdb_rating': imdb_ratings,
                      'metascore': metascores,
                      'votes': votes,
                      'descriptions':descriptions}
                      )

# final_df = movies2022.loc[movies2022['year'] != 'ovie']
# movies2022.loc[:, 'year'] = pd.to_numeric(movies2022['year'])
# print(movies2022.iloc[2,:])
movies2022['descriptions'] = movies2022['descriptions'].replace(r"\n", "", regex=True)
movies2022.to_csv('movies2022.csv', index=False)
